In [19]:
import os
from IPython.display import Image
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

### copy the fuctions from testrun & change the return value to fit other ones's functions

In [20]:
def tail(f, lines=1, _buffer=4098):
    """Tail a file and get X lines from the end"""
    # place holder for the lines found
    lines_found = []

    # block counter will be multiplied by buffer
    # to get the block size from the end
    block_counter = -1

    # loop until we find X lines
    while len(lines_found) < lines:
        try:
            f.seek(block_counter * _buffer, os.SEEK_END)
        except IOError:  # either file is too small, or too many lines requested
            f.seek(0)
            lines_found = f.readlines()
            break

        lines_found = f.readlines()

        # decrement the block counter to get the
        # next X bytes
        block_counter -= 1

    return lines_found[-lines:]

In [21]:
def get_bond_orders(filename):
    """Fetches the bond indices from the end of the JANPA output."""
    f = open(filename)
    lines = tail(f, lines=56)
    f.close()

    split_lines = [line.split() for line in lines[:-9]]

    for line in split_lines:
        del line[0:2]
        line[0] = line[0][:-1]

    n_atoms = len(split_lines)
    bond_orders = np.zeros((n_atoms, n_atoms))

    indices = np.triu_indices(47)

    bond_orders[indices] = [float(item) for sublist in split_lines for item in sublist]
    return bond_orders

In [22]:
def convert_bond_orders(bond_orders):
    """Round bond orders to their nearest bond type"""
    for i in range(bond_orders.shape[0]):
        for j in range(bond_orders.shape[1]):
            if np.isclose(bond_orders[i, j], 1.5, atol=0.15):
                bond_orders[i, j] = 1.5
            else:
                bond_orders[i, j] = int(round(bond_orders[i, j]))
    bonding_matrix = bond_orders
    return bonding_matrix

### test the fuctions

In [23]:
bond_orders = get_bond_orders('output.janpa')
bond_orders

array([[4.0091e+00, 1.4487e+00, 1.2800e-02, ..., 1.0000e-04, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 4.0095e+00, 1.4575e+00, ..., 0.0000e+00, 1.0000e-04,
        1.0000e-04],
       [0.0000e+00, 0.0000e+00, 3.9965e+00, ..., 1.0000e-04, 0.0000e+00,
        0.0000e+00],
       ...,
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 9.4380e-01, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 9.6220e-01,
        6.0000e-04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        9.5850e-01]])

In [24]:
bonding_matrix = convert_bond_orders(bond_orders)
bonding_matrix

array([[4. , 1.5, 0. , ..., 0. , 0. , 0. ],
       [0. , 4. , 1.5, ..., 0. , 0. , 0. ],
       [0. , 0. , 4. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 1. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 1. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 1. ]])

In [25]:
with open("bond_order.txt", 'r+') as file:
    for i in range(bonding_matrix.shape[0]):
        for j in range(bonding_matrix.shape[1]):
            file.write(str(bonding_matrix[i, j]))

FileNotFoundError: [Errno 2] No such file or directory: 'bond_order.txt'

### write the test fuctions

In [26]:
def test_convert_bond_orders():
    bond_orders = get_bond_orders('output.janpa')##should be modified
    bonding_matrix = convert_bond_orders(bond_orders)
    assert bonding_matrix.shape[0] == bonding_matrix.shape[1], "the bond orders matrix should be square"
    assert bonding_matrix[2][2] > 0, "the diagonal of the bond orders matrix should be positive"
    return

In [27]:
assert -1 > 0, "the diagonal of the bond orders matrix should be positive"

AssertionError: the diagonal of the bond orders matrix should be positive

### nosetests

In [28]:
cd tests

[WinError 2] The system cannot find the file specified: 'tests'
C:\Users\sxzxc\Desktop\blues\blues\tests


In [30]:
%%cmd
nosetests test_bondorders.py

Microsoft Windows [Version 10.0.17134.48]
(c) 2018 Microsoft Corporation. All rights reserved.

(my-rdkit-env) C:\Users\sxzxc\Desktop\blues\blues\tests>nosetests test_bondorders.py

(my-rdkit-env) C:\Users\sxzxc\Desktop\blues\blues\tests>

.
----------------------------------------------------------------------
Ran 1 test in 0.057s

OK
